## Transfer Scene Viewing Eyetracking Files to Compute Canada (.ps1)

VERSION: `Scene_Eyetrack_Auto_Sync_General.ps1`

### Advanced Explanation

This PowerShell script automates the transfer of directories from a local machine to a remote server based on user-defined criteria. It starts by fetching the current date and listing all local directories. The user is prompted to enter their username and confirm or specify the remote path, which is constructed using `/home/$username/scratch/Actual_Final_SceneViewing_EyeTracking`.

The script iterates over each local directory, filtering subdirectories created in the last 24 hours. For each recent directory, it calculates the size in MB and fetches the creation time. The user is then prompted to confirm the transfer of each directory to the server. On confirmation, `scp` (Secure Copy Protocol) is used to transfer the directory to the constructed remote path. Success or failure of each transfer is logged, and the number of successfully transferred directories is tracked and output at the end.

### User-Friendly Explanation

This script helps you transfer recent folders from your computer to a server. First, it asks for your username and checks the server path where the folders will be transferred. You can confirm or correct this path. Then, it looks for folders created in the last 24 hours and asks you for each folder if you want to transfer it to the server.

For each folder you choose, the script calculates its size and notes when it was created. If you say yes, it transfers the folder to the server using your username and the confirmed path. After the process, the script tells you how many folders were transferred and their names. This is a useful way to back up new folders to a server regularly.

### Oversimplified Explanation

This should be kept and run where scene viewing eyetracking session folders are backed up. It will send new ones from the local device to your scratch folder on Compute Canada. 

### Note
Ensure that all 6 session folders match after sending. 
    - (4 Encoding, 3 Retrieval)
    
Track meta data before sending and compare it to the output of the check scripts. 

In [ ]:
$currentDate = Get-Date
$localFolders = Get-ChildItem -Path . -Directory

$transferredCount = 0
$transferredDirectories = @()

$username = Read-Host "Please enter your username"

$remotePathBase = "/home/$username/scratch/Actual_Final_SceneViewing_EyeTracking"

Write-Output "The constructed remote path is: $remotePathBase"
$pathConfirmation = Read-Host "Is this path correct? [Y/N]"

if ($pathConfirmation -ne "Y") {
    $remotePathBase = Read-Host "Please enter the full correct remote path"
}

foreach ($localFolder in $localFolders) {

    $recentDirectories = Get-ChildItem -Path $localFolder.FullName -Directory | Where-Object {
        ($currentDate - $_.CreationTime).TotalHours -le 24
    }

    foreach ($dir in $recentDirectories) {

        $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
        $creationTime = $dir.CreationTime
        $userInput = Read-Host "Do you want to transfer $($dir.Name) [Size: $($directorySize) MB, Created: $($creationTime)] to the server at `"$remotePathBase`"? [Y/N]"

        if ($userInput -eq "Y") {
            $remotePath = "$remotePathBase/$($localFolder.Name)/$($dir.Name)"

            scp -r $dir.FullName "$username@graham.computecanada.ca:`"$remotePath`""
            if ($?) {
                $transferredCount++
                $transferredDirectories += $dir.Name
                
                Write-Output ("Directory '{0}' transferred to '{1}'. Size: {2:N2} MB" -f $dir.Name, $remotePath, $directorySize)
            } else {
                Write-Warning "Failed to transfer directory: $dir"
            }
        }
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}


VERSION: `Scene_Eyetrack_Auto_Sync_(name).ps1`

Replace remote pathbase and scp command to reflect the user and save with your name using Powershell ISE. Note that there will be no prompts as the script is designed for maximum efficiency so make sure you know what you are doing. 

In [ ]:
$currentDate = Get-Date
$localFolders = Get-ChildItem -Path . -Directory

$transferredCount = 0
$transferredDirectories = @()

$remotePathBase = "/home/samhsu03/scratch/FinalSceneViewingEyetracking/"

foreach ($localFolder in $localFolders) {

    $recentDirectories = Get-ChildItem -Path $localFolder.FullName -Directory | Where-Object {
        ($currentDate - $_.CreationTime).TotalHours -le 24
    }

    foreach ($dir in $recentDirectories) {

        $remotePath = "$remotePathBase/$($localFolder.Name)/$($dir.Name)"

        scp -r $dir.FullName "samhsu03@graham.computecanada.ca:`"$remotePath`""
        if ($?) {
            $transferredCount++
            $transferredDirectories += $dir.Name
            
            $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
            Write-Output ("Directory '{0}' transferred to '{1}'. Size: {2:N2} MB" -f $dir.Name, $remotePath, $directorySize)
        } else {
            Write-Warning "Failed to transfer directory: $dir"
        }
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}
